In [1]:
import sys
sys.path.append('..')

# Helper imports
from data import CIFAR10, IMAGENETTE
import utils

# Numerical computing and display imports
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

# Other imports
from datetime import datetime
import os

/Users/neddamj/miniforge3/envs/nhash/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Load and Prepare the Data
folder_path = '../../images/'
dataset = 'imagenette'    
if dataset == 'cifar10':
    images = CIFAR10()
if dataset == 'imagenette':
    images = IMAGENETTE()
x = images.load()
images.save_to_disk(x, folder_path, num_images=100)
folder_path = utils.move_data_to_temp_ram(folder_path, ram_size_mb=50)
del(x)

Metal device set to: Apple M1

systemMemory: 8.00 GB
maxCacheSize: 2.67 GB



2024-02-19 15:46:16.080097: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


[INFO] Images saved
Started erase on disk4
Unmounting disk
Erasing
Initialized /dev/rdisk4 as a 50 MB case-insensitive HFS Plus volume
Mounting disk
Finished erase on disk4 (TempRAM)
[INFO] RAM Disk Created...
[INFO] Files moved to temp RAM...


In [3]:
attack_hamming_threshold = int(0.4 * 127 + 0.5)

In [4]:
# Threshold-hyperparameter mappings
threshold_hyperparam = {
    '1':  {'simba_hamming_threshold': 7,  'simba_l2_threshold': 20, 'hsja_l2_threshold': 8},
    '6':  {'simba_hamming_threshold': 10, 'simba_l2_threshold': 20, 'hsja_l2_threshold': 12},
    '13': {'simba_hamming_threshold': 20, 'simba_l2_threshold': 28, 'hsja_l2_threshold': 15},
    '25': {'simba_hamming_threshold': 25, 'simba_l2_threshold': 30, 'hsja_l2_threshold': 24},
    '38': {'simba_hamming_threshold': 40, 'simba_l2_threshold': 35, 'hsja_l2_threshold': 24},
    '51': {'simba_hamming_threshold': 55, 'simba_l2_threshold': 35, 'hsja_l2_threshold': 30}
}


# SIMBA Hyperparams
simba_epsilon = 0.9
simba_hamming_threshold = threshold_hyperparam[str(attack_hamming_threshold)]['simba_hamming_threshold']                                # <= 5% - 10(max) & 10% - 20 & 20% - 23 
simba_l2_threshold = threshold_hyperparam[str(attack_hamming_threshold)]['simba_l2_threshold']
simba_max_steps = 1000
fast = True

# HSJA Hyperparams
hsja_max_steps = 10
hsja_grad_queries = 20
hsja_l2_threshold = threshold_hyperparam[str(attack_hamming_threshold)]['hsja_l2_threshold']

# Other params
ssim_threshold = 0.8
now = datetime.now()
dt_string = now.strftime("%d-%m-%Y_%H:%M:%S")

In [5]:
# Threshold-hyperparameter mappings
nes_threshold_hyperparam = {
    '1':  {'nes_l2_threshold': 25, 'nes_l2_tolerance': 10, 'hsja_l2_threshold': 10},
    '6':  {'nes_l2_threshold': 35, 'nes_l2_tolerance': 12, 'hsja_l2_threshold': 15},
    '13': {'nes_l2_threshold': 45, 'nes_l2_tolerance': 15, 'hsja_l2_threshold': 20},
    '25': {'nes_l2_threshold': 55, 'nes_l2_tolerance': 20, 'hsja_l2_threshold': 24},
    '38': {'nes_l2_threshold': 75, 'nes_l2_tolerance': 10, 'hsja_l2_threshold': 24},
    '51': {'nes_l2_threshold': 75, 'nes_l2_tolerance': 10, 'hsja_l2_threshold': 24}
}

# NES Hyperparams
nes_mean =  -0.025 
nes_std = 0.1
nes_sigma = 0.7
nes_eps = 0.1                      
nes_hamming_threshold = nes_threshold_hyperparam[str(attack_hamming_threshold)]['nes_l2_threshold'] 
nes_l2_tolerance = nes_threshold_hyperparam[str(attack_hamming_threshold)]['nes_l2_tolerance'] 
nes_num_samples = 50

In [6]:
zo_threshold_hyperparam = {
    '1':  {'zo_l2_threshold': 25, 'zo_l2_tolerance': 10},
    '6':  {'zo_l2_threshold': 35, 'zo_l2_tolerance': 12},
    '13': {'zo_l2_threshold': 45, 'zo_l2_tolerance': 15},
    '25': {'zo_l2_threshold': 55, 'zo_l2_tolerance': 20},
    '38': {'zo_l2_threshold': 75, 'zo_l2_tolerance': 10},
    '51': {'zo_l2_threshold': 75, 'zo_l2_tolerance': 10}
}  

# ZO-SignSGD Hyperparameters
zo_max_queries = 50
zo_epsilon = 0.1
zo_upper_tolerance = 10 
zo_lower_tolerance = 5
zo_l2_threshold = zo_threshold_hyperparam[str(attack_hamming_threshold)]['zo_l2_threshold'] 
zo_l2_tolerance = zo_threshold_hyperparam[str(attack_hamming_threshold)]['zo_l2_tolerance'] 
zo_search_steps = 50

In [8]:
# Soft-Label Attack imports
from zo_signsgd import ZOSignSGDttack
from simba import SimBAttack
from nes import NESAttack

zosignsgd = ZOSignSGDttack(max_queries=zo_max_queries, 
                           epsilon=zo_epsilon, 
                           hamming_threshold=attack_hamming_threshold,
                           search_steps=zo_search_steps)
simba = SimBAttack(eps=simba_epsilon,
                   hamming_threshold=simba_hamming_threshold,
                   max_steps=simba_max_steps,
                   fast=fast)
nes = NESAttack(mean=nes_mean,
                std=nes_std,
                sigma=nes_sigma,
                eps=nes_eps,
                hamming_threshold=attack_hamming_threshold,
                num_samples=nes_num_samples)

In [9]:
# Hard-label Attack imports
from hsja import HSJAttack

hsja = HSJAttack(max_iters=hsja_max_steps, 
                 grad_queries=hsja_grad_queries, 
                 l2_threshold=hsja_l2_threshold, 
                 hamming_threshold=attack_hamming_threshold)

In [10]:
# Joint Attack
from joint_attack import JointAttack

joint_attack = JointAttack(soft_label=zosignsgd, 
                           hard_label=hsja)

In [11]:
for i in range(99):
    # Format the path to the input image
    img_path = f'../../images/{i+2}.bmp' 
    target_path = f'../../images/{1}.bmp'
    _, _, _, _, path, filetype = img_path.split('.')
    img_path = path.split('/')
    img_path = f'{folder_path}{img_path[2]}.{filetype}'

    # Load the target image
    target_img = utils.load_img(target_path)

    # Attack NeuralHash 
    orig_img, sl_img, adv_img, sl_num_queries, hl_num_queries = joint_attack.attack(img_path, target_path)
    
    # Save the simba image and the final image
    utils.save_img(f'../../images/{i+2}_sl.bmp', sl_img)  
    utils.save_img(f'../../images/{i+2}_final.bmp', adv_img)

    # Attack Metrics
    orig_hash  = utils.compute_hash(orig_img)
    target_hash = utils.compute_hash(target_path)
    sl_hash = utils.compute_hash(sl_img)
    adv_hash = utils.compute_hash(adv_img)
    simba_hamming_dist = utils.distance(target_hash, sl_hash, "hamming")
    final_hamming_dist = utils.distance(target_hash, adv_hash, "hamming")
    sl_l2_dist = utils.distance(orig_img, sl_img, 'l2')
    final_l2_dist = utils.distance(orig_img, adv_img, 'l2')
    sl_ssim = utils.distance(orig_img, sl_img, 'ssim')
    final_ssim = utils.distance(orig_img, adv_img.astype(np.uint8), 'ssim')
    total_queries = sl_num_queries + hl_num_queries
    success = (final_hamming_dist <= attack_hamming_threshold) and (final_ssim >= ssim_threshold)

    attack_metrics = {
        'Image Path':         [img_path],
        'Target Path':        [target_path],
        'Success':            [success],
        'Queries':            [total_queries],
        'Soft-Label L2':      [sl_l2_dist],
        'Final L2':           [final_l2_dist],
        'Sotf-Label SSIM':    [sl_ssim],
        'Final SSIM':         [final_ssim],
        'Soft-Label Hamming': [simba_hamming_dist],
        'Final Hamming':      [final_hamming_dist]
    }

    # Save the results
    df = pd.DataFrame.from_dict(attack_metrics)
    if type(joint_attack.soft_label) == SimBAttack:
        sl_attack = 'simba'
        eps = simba_epsilon
    elif type(joint_attack.soft_label) == NESAttack:
        sl_attack = 'nes'
        eps = nes_eps
    else:
        sl_attack = 'zo_signsgd'
        eps = zo_epsilon
    file_path = f'metrics/{sl_attack}/{eps}/hamm_{attack_hamming_threshold}_l2_{hsja_l2_threshold}_{dt_string}.csv'
    if os.path.exists(file_path):
        df.to_csv(file_path, mode='a', index=False, header=False)
    else: 
        df.to_csv(file_path, index=False, header=True)  

Estimating Gradients...
Gradient Estimation Complete...
Adding 0.1 noise
Step: 1 L2 Distance: 1.0747459308325924
HSJA Iteration: 1
[INFO] Starting Boundary Search...
Hamming Dist: 52
L2 Dist: 207.08789732008054
Hamming Dist: 47
L2 Dist: 207.08789732008054
Hamming Dist: 41
L2 Dist: 215.73284330154354
Hamming Dist: 47
L2 Dist: 227.61961947772164
Hamming Dist: 45
L2 Dist: 235.17292995268852
Hamming Dist: 50
L2 Dist: 239.3099766088126
Hamming Dist: 51
L2 Dist: 241.46331267830388
Hamming Dist: 50
L2 Dist: 242.56053157540063
Hamming Dist: 50
L2 Dist: 243.11419796347056
Hamming Dist: 52
L2 Dist: 243.39228534981854
Hamming Dist: 50
L2 Dist: 243.39228534981854
Hamming Dist: 50
L2 Dist: 243.46193735711498
Hamming Dist: 52
L2 Dist: 243.49678285414754
Hamming Dist: 50
L2 Dist: 243.49678285414754
Hamming Dist: 52
L2 Dist: 243.5054962580348
Hamming Dist: 50
L2 Dist: 243.5054962580348
Hamming Dist: 52
L2 Dist: 243.50767473582644
Hamming Dist: 50
L2 Dist: 243.50767473582644
Hamming Dist: 52
L2 Dist: 2

OSError: Cannot save file into a non-existent directory: 'metrics/zo_signsgd/0.1'

In [ ]:
def plot_results(orig_img, sl_img, adv_img, hamming_threshold):
    fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(15,6))
    #fig.suptitle(f'Hamming Threshold: {hamming_threshold}')
    ax1.imshow(orig_img)
    ax1.set_title('Orginal Image') 
    ax2.imshow(sl_img)
    ax2.set_title('Noisy Soft-Label Image')
    ax3.imshow(adv_img.astype(np.uint8))
    ax3.set_title('Denoised Final Image')

In [ ]:
# Show the images/results
plot_results(orig_img, sl_img, adv_img, attack_hamming_threshold)

### Analyze the Results

In [ ]:
df = pd.read_csv(file_path)
df.head()

In [ ]:
queries = np.array(df['Queries'])
avg_queries = queries.mean()
print(f'The attacks were completed in {int(avg_queries)} queries on average')

In [ ]:
success = np.array(df['Success'])
ASR = success.mean()
print(f'The ASR is {100*ASR:.2f}% ')

In [ ]:
hamming_dist = np.array(df['Final Hamming'])
avg_hamm = hamming_dist.mean()
print(f'The average hamming distance after joint attack is {avg_hamm}')

In [ ]:
simba_l2_dist = np.array(df['Soft-Label L2'])/np.sqrt(224*224*3)
simba_avg_l2 = simba_l2_dist.mean()
simba_l2_std = np.std(simba_l2_dist)
print(f'The average L2 distortion after the soft-label attack is {simba_avg_l2:.4f} +- {simba_l2_std:.4f}')

final_l2_dist = np.array(df['Final L2'])/np.sqrt(224*224*3)
final_avg_l2 = final_l2_dist.mean()
final_l2_std = np.std(final_l2_dist)
print(f'The average L2 distortion after the complete attack is {final_avg_l2:.4f} +- {final_l2_std}')

In [ ]:
index = np.arange(100)
bar_width = 0.3
fig, ax = plt.subplots(figsize=(15,6)) 
simba_distortion = ax.bar(index, df['Soft-Label L2'], bar_width, label='Soft-Label L2 Dist')
output_distortion = ax.bar(index + bar_width, df['Final L2'], bar_width, label='Final L2 Dist')
ax.set_label('Image')
ax.set_ylabel('L2 Distortion')
ax.legend()

plt.show()